# How do delays relate to traffic volume and airport performances? 

We need to analyze first:
- Delay rate (calculate from df_AID): total delays/total flights per airport per month
- Traffic volume: total flights 
And then deep dive into aiport performances:  
- Efficiencies: consistency coefficient (from Q2), taxi in & out efficiency (TOTAL_REF_TIME_MIN + TOTAL_ADD_TIME_MIN)/VALID_FL, Departure Punctuality %, Arrival Punctuality % 

In [ ]:
#Importing all libraries needed
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

In [ ]:
#Datasets used
df_AID= pd.read_excel("datasets/Airport_IATA_delays_airline_reported.xlsx")
df_AT = pd.read_excel("datasets/AirportTraffic.xlsx")
df_TIn = pd.read_excel("datasets/Taxi-In_Additional_Time.xlsx")
df_TOut = pd.read_excel("datasets/Taxi-Out_Additional_Time.xlsx")
df_AP = pd.read_excel("datasets/Airports_Punctuality.xlsx")

In [ ]:
df_top20 = (
    df_AT.groupby("APT_ICAO")[["FLT_TOT_1", "FLT_DEP_1", "FLT_ARR_1"]] #Group by airport code
    .sum().sort_values(by="FLT_TOT_1",ascending=False) #Sum the values for each code of the 3 columns indicated
    .head(20).reset_index()) #Change "20" to change the number of airports analysed

# Adding airport's city name and state from original dataset
df_top20 = (df_top20.merge(df_AT[["APT_ICAO", "APT_NAME", "STATE_NAME"]]
                           .drop_duplicates(), on="APT_ICAO", how="left"))
airports_code_list = df_top20["APT_ICAO"].tolist()

Let's find the delay rate as the ratio between total delayed flights and total flights, for each of the 20 airports, in every month between 2023 and 2024

In [ ]:
# Filter the delay dataset for only the top 20 airports
df_AID_TOP20 = df_AID[df_AID["APT_ICAO"].isin(airports_code_list)].copy()

# Calculate delay ratio as percentage
df_AID_TOP20["Delay_Ratio"] = (df_AID_TOP20["TF"] / df_AID_TOP20["Total_Flights_Period"]) * 100

# Provide key insights per year per month per airport
df_AID_TOP20_YMA = (
    df_AID_TOP20
    .groupby(["Year_Lobt","Month_Lobt","APT_ICAO"], as_index=False)
    .agg({
        "TD": "sum",
        "TF": "sum",
        "Total_Flights_Period": "sum",
        "adm": "mean",
        "pd": "mean",
        "Delay_Ratio": "mean" 
    })
    .sort_values(by="TF", ascending=False)
    .rename(columns={
        "TD": "Total Delay (TD)",
        "TF": "Total Flights (TF)",
        "Total_Flights_Period": "Total_Flights_Period",
        "adm": "Avg Delay per Movement (in min)",
        "pd": "Avg Proportion of Delay (%)",
        "Delay_Rate": "Avg Delay Ratio (%)"
    })
    .reset_index(drop=True)
    .sort_values(["APT_ICAO", "Year_Lobt"])
)
display(df_AID_TOP20_YMA)

This dataframe contains both the delay rate (Delay_Ratio) and the total flights (Total_Flights_Period).
For the performances we consider:
- consistency coefficient (from Q2) 
- taxi in & out efficiency (TOTAL_REF_TIME_MIN + TOTAL_ADD_TIME_MIN)/VALID_FL
- Departure and Arrival Punctuality % 

In [82]:
'''Consistency coefficient code missing'''

'Consistency coefficient code missing'

Taxis efficiencies: import and cleaning datasets

In [12]:
df_TIn = pd.read_excel("datasets/Taxi-In_Additional_Time.xlsx")
df_TOut = pd.read_excel("datasets/Taxi-Out_Additional_Time.xlsx")

In [ ]:
df_taxi_time_in = (
    df_TIn[(df_TIn["APT_ICAO"].isin(airports_code_list)) & 
           ((df_TIn["YEAR"] == 2023) | (df_TIn["YEAR"] == 2024))]
    .groupby(["APT_ICAO", "YEAR"])[["VALID_FL", "TOTAL_REF_NB_FL", "TOTAL_REF_TIME_MIN", "TOTAL_ADD_TIME_MIN"]]
    .sum()
    .reset_index()
)
#In order: filter only airports in top 20, filter only values from 2023-24, group by code and sum the values in the 
#listed columns, reset the index to have the codes as a column itselft (useful for merging)

df_airport_codes = pd.DataFrame({'APT_ICAO': airports_code_list})
# Merging taxi with airport codes
df_taxi_time_in = pd.merge(df_airport_codes, df_taxi_time_in, on='APT_ICAO', how='left')
df_taxi_time_in["TI_EFF"] = (df_taxi_time_in["TOTAL_REF_TIME_MIN"] + df_taxi_time_in["TOTAL_ADD_TIME_MIN"])/df_taxi_time_in["VALID_FL"] #Minutes per flight
df_taxi_time_in.head()

In [ ]:
df_taxi_time_out = (
    df_TOut[(df_TOut["APT_ICAO"].isin(airports_code_list)) & 
           ((df_TOut["YEAR"] == 2023) | (df_TOut["YEAR"] == 2024))]
    .groupby(["APT_ICAO", "YEAR"])[["VALID_FL", "TOTAL_REF_NB_FL", "TOTAL_REF_TIME_MIN", "TOTAL_ADD_TIME_MIN"]]
    .sum()
    .reset_index()
)

# Merging taxi in with taxi out
df_taxi_time_out = pd.merge(df_airport_codes, df_taxi_time_out, on='APT_ICAO', how='left').dropna()
df_taxi_time_out["TO_EFF"] = (df_taxi_time_out["TOTAL_ADD_TIME_MIN"] + df_taxi_time_out["TOTAL_REF_TIME_MIN"])/df_taxi_time_out["VALID_FL"] #Minutes per flight
df_taxi_time_out.head()

Merging the datasets and removing unnecessary columns 

In [ ]:
df_tt_efficiencies = pd.merge(df_taxi_time_out, df_taxi_time_in, on=["APT_ICAO", "YEAR"], how="left")
tt_cols_todrop = ["VALID_FL_x", "TOTAL_REF_NB_FL_x", "TOTAL_REF_TIME_MIN_x", "TOTAL_ADD_TIME_MIN_x", "VALID_FL_y", "TOTAL_REF_NB_FL_y", "TOTAL_REF_TIME_MIN_y", "TOTAL_ADD_TIME_MIN_y"]
df_tt_efficiencies = df_tt_efficiencies.drop(columns=tt_cols_todrop, axis=1)
df_tt_efficiencies.head()

Depart & Arrival Punctualities

In [ ]:
airport_map = {
    "LTFM": "Istanbul",
    "EHAM": "Amsterdam",
    "EGLL": "London Heathrow",
    "LFPG": "Paris Charles de Gaulle",
    "EDDF": "Frankfurt",
    "LEMD": "Madrid Barajas",
    "LEBL": "Barcelona",
    "EDDM": "Munich",
    "LIRF": "Rome Fiumicino",
    "EGKK": "London Gatwick",
    "LSZH": "Zurich",
    "LGAV": "Athens",
    "EIDW": "Dublin",
    "LOWW": "Vienna",
    "LEPA": "Palma de Mallorca",
    "EKCH": "Copenhagen",
    "LPPT": "Lisbon",
    "LTFJ": "Istanbul Sabiha Gokcen",
    "LTAI": "Antalya",
    "ENGM": "Oslo"
} #Dataset has different names from the ones used before

In [ ]:
top20_codes = [airport_map[a] for a in df_top20['APT_ICAO'] if a in airport_map]
df_AP_top20 = df_AP[df_AP['Airport'].isin(top20_codes)]

name_to_icao = {name: icao for icao, name in airport_map.items()}
icao_codes = df_AP_top20["Airport"].map(name_to_icao)
df_AP_top20.insert(loc=2, column="APT_ICAO", value=df_AP_top20["Airport"].map(name_to_icao))

df_AP_top20.head()

Adding the month and year column from the date one

In [ ]:
df_AP_top20["Year"] = df_AP_top20["Date"].dt.year.astype(str)
df_AP_top20["Month"] = df_AP_top20["Date"].dt.month.apply(lambda x: f'{x:02d}')

Filtering the dataset and dropping unnecessary columns

In [81]:
AP_cols_todrop = ["Avg Departure Schedule Delay", "Avg Arrival Schedule Delay", "Avg Departure - Arrival Schedule Delay", "Operated Schedules %"]
df_AP_efficiencies = (df_AP_top20[df_AP_top20["Date"].dt.year.astype(str).isin(["2023", "2024"])]
                      .groupby(["APT_ICAO", "Year", "Month"])
                      .mean(["Departure Punctuality %", "Arrival Punctuality %"])
                      .reset_index()).drop(columns=AP_cols_todrop, axis=1)
df_AP_efficiencies 

,APT_ICAO,Year,Month,Departure Punctuality %,Arrival Punctuality %
0,EDDF,2023,01,0.667677,0.784448
1,EDDF,2023,02,0.636383,0.766183
2,EDDF,2023,03,0.532972,0.701877
3,EDDF,2023,04,0.659708,0.754957
4,EDDF,2023,05,0.589679,0.602905
...,...,...,...,...,...
475,LTFM,2024,08,0.636076,0.671603
476,LTFM,2024,09,0.680400,0.713811
477,LTFM,2024,10,0.778200,0.783877
478,LTFM,2024,11,0.821263,0.825065


Now that we have some efficiencies:
- plot in the same chart the delay rate, traffic flow, effieciency trends through the years and see if there are some patterns 